# Books : Visualisation and recommendation
[Kaggle | Books : Visualisation and recommendation](https://www.kaggle.com/code/nayansakhiya/books-visualisation-and-recommendation)   
[Kaggle | Bookcrossing EDA & Visualization (Plotly)](https://www.kaggle.com/code/sanzharsamatuly/bookcrossing-eda-visualization-plotly)
### DataSet 
- [goodbooks-10k](https://www.kaggle.com/datasets/zygmunt/goodbooks-10k)
- [Book-Crossing: User review ratings](https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset?select=Book+reviews)
---
# 캐굴캐굴 필사 스터디 1주차
#### 1주차 학습 기간 : 23.01.28 ~ 23.02.04   
주제 : EDA    
해당 데이터 셋에서 책 제목, 저자, 출판사, 사용자 및 등급에 대한 정보를 확인하여 사용자가 어떤 책을 읽고 있는지 인사이트를 얻어낸다.     
1. About Dataset
2. Data Visualization 
    
## 복습 
2023.03.26 
- KNN을 이용한 책 추천 시스템 (커널 필사) 

## Import Libraries

In [2]:
!pip install plotly_express

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import pandas as pd
import os
import re
import string

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
import plotly_express as px
%matplotlib inline


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


- pandas : data processing, CSV file I/O
- numpy : 선형대수

In [5]:
import os

root = f"/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/"
data_dir = "dataset"
for (root, dirs, files) in os.walk(root+data_dir):
    print("# root : " + root)
    if len(dirs) > 0:
        for dir_name in dirs:
            print("dir: " + dir_name)

    if len(files) > 0:
        for file_name in files:
            print("file: " + file_name)

# root : /content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset
dir: summary
dir: reviews
dir: goodbooks-10
# root : /content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/summary
file: Preprocessed_data.csv
# root : /content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/reviews
file: BX-Book-Ratings.csv
file: BX-Users.csv
file: BX_Books.csv
# root : /content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/goodbooks-10
file: book_tags.csv
file: books.csv
file: ratings.csv
file: tags.csv
file: to_read.csv


In [6]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/summary/Preprocessed_data.csv
/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/reviews/BX-Book-Ratings.csv
/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/reviews/BX-Users.csv
/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/reviews/BX_Books.csv
/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/goodbooks-10/book_tags.csv
/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/goodbooks-10/books.csv
/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/goodbooks-10/ratings.csv
/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/goodbooks-10/tags.csv
/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/goodbooks-10/to_read.csv


In [7]:
from glob import glob

print(glob('/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/*'))

['/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/summary', '/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/reviews', '/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/goodbooks-10']


### Read .csv data 

In [8]:
# goodreads data
goodbooks_path = '/content/drive/MyDrive/캐굴캐굴/goodbooks-10k/dataset/goodbooks-10/'
books_data = pd.read_csv(f'{goodbooks_path}/books.csv',error_bad_lines = False)
tags_data = pd.read_csv(f'{goodbooks_path}/book_tags.csv')
ratings_data = pd.read_csv(f'{goodbooks_path}/ratings.csv')
book_tags = pd.read_csv(f'{goodbooks_path}/tags.csv')

# book crossing data
book_path = '/content/drive/MyDrive/캐굴캐굴/goodbooks-10k//dataset/reviews'

user_cols = ['user_id', 'location', 'age']
cross_users_data = pd.read_csv(f'{book_path}/BX-Users.csv', sep=';', names=user_cols, encoding='latin-1', low_memory=False, skiprows=1)

book_cols = ['isbn','book_title' ,'book_author','year_of_publication', 'publisher', 'img_s', 'img_m', 'img_l']
cross_books_data = pd.read_csv(f'{book_path}/BX_Books.csv', sep=';', names=book_cols, encoding='latin-1', low_memory=False, skiprows=1)

rating_cols = ['user_id', 'isbn', 'rating']
cross_ratings_data = pd.read_csv(f'{book_path}/BX-Book-Ratings.csv', sep=';', names=rating_cols, encoding='latin-1', low_memory=False, skiprows=1)

<ipython-input-8-5747fa46a06c>:3: FutureWarning:

The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.





In [9]:
books_data.head()

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [10]:
cross_users_data.head()

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [11]:
cross_ratings_data.head()

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [12]:
cross_books_data.head()

,isbn,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton & Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


### Drop unnecessary data

In [13]:
books_data = books_data.drop(columns=['id', 'best_book_id', 'work_id', 'isbn', 'isbn13', 'title','work_ratings_count',
                                   'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 
                                    'image_url','small_image_url'])

In [14]:
books_data = books_data.dropna()
cross_books_data = cross_books_data.drop(columns=['img_s', 'img_m', 'img_l'])

### Drop Duplicates from all the dataset


In [15]:
ratings_data = ratings_data.sort_values("user_id")

ratings_data.drop_duplicates(subset =["user_id","book_id"], keep = False, inplace = True) 
books_data.drop_duplicates(subset='original_title',keep=False,inplace=True)
book_tags.drop_duplicates(subset='tag_id',keep=False,inplace=True)
tags_data.drop_duplicates(subset=['tag_id','goodreads_book_id'],keep=False,inplace=True)
cross_ratings_data.drop_duplicates(subset =["user_id","isbn"], keep = False, inplace = True) 
cross_books_data.drop_duplicates(subset='book_title',keep=False,inplace=True)

### Clean the text

In [16]:
def clean_text(text):
    '''Make text lowercase,
       remove text in square brackets,
       remove links,remove punctuation
       and remove words containing numbers.
    '''
    
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [17]:
cross_books_data['book_title'] = cross_books_data['book_title'].apply(lambda x:clean_text(x))

In [18]:
merge_data = pd.merge(cross_books_data, cross_ratings_data, on='isbn')
merge_data = merge_data.sort_values('isbn', ascending=True)
merge_data.head()

,isbn,book_title,book_author,year_of_publication,publisher,user_id,rating
626418,0000913154,the way things work an illustrated encyclopedi...,C. van Amerongen (translator),1967,Simon & Schuster,171118,8
587195,0001010565,mogs christmas,Judith Kerr,1992,Collins,209516,0
587194,0001010565,mogs christmas,Judith Kerr,1992,Collins,86123,0
441049,0001046713,twopence to cross the mersey,Helen Forrester,1992,HarperCollins Publishers,196149,0
263949,000104687X,ts eliot reading the wasteland and other poems,T.S. Eliot,1993,HarperCollins Publishers,23902,6


In [19]:
books_data.head()

,book_id,books_count,authors,original_publication_year,original_title,language_code,average_rating,ratings_count
0,2767052,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653
1,3,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,eng,4.44,4602479
3,2657,487,Harper Lee,1960.0,To Kill a Mockingbird,eng,4.25,3198671
4,4671,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,eng,3.89,2683664
5,11870085,226,John Green,2012.0,The Fault in Our Stars,eng,4.26,2346404


## Data Visualization 
### Good Reads Visualisation
- goodbooks dataset 설명 

```
There have been good datasets for movies (Netflix, Movielens) and music (Million Songs) recommendation, but not for books. That is, until now.

This dataset contains ratings for ten thousand popular books. As to the source, let's say that these ratings were found on the internet. Generally, there are 100 reviews for each book, although some have less - fewer - ratings. Ratings go from one to five.

Both book IDs and user IDs are contiguous. For books, they are 1-10000, for users, 1-53424. All users have made at least two ratings. Median number of ratings per user is 8.

There are also books marked to read by the users, book metadata (author, year, etc.) and tags.
```

In [20]:
import plotly
plotly.__version__
# 1.12.12

'5.13.1'

In [21]:
import plotly_express as px

In [22]:
top_rated = books_data.sort_values('average_rating', ascending=False)
top30 = top_rated[:30]

fig = px.bar(top30, x="average_rating", y="original_title", title='Top 30 Rated Books and Their Ratings',
             orientation='h', color='original_title', width=1500, height=700)
fig.show(renderer="colab")

In [23]:
fig = px.treemap(top30, path=['original_title'], values='average_rating',title='Top 30 Rated Books', width=1000, height=700)
fig.show(renderer="colab")

In [24]:
top_popular = books_data.sort_values('ratings_count', ascending=False)
top30_book = top_popular[:30]
fig = px.bar(top30_book, x="ratings_count", y="original_title", title='Top 30 Popular Books', orientation='h', color='original_title',
             width=1500, height=700)
fig.show(renderer="colab")

In [25]:
fig = px.treemap(top30_book, path=['original_title'], values='ratings_count',title='Top 30 Popular Books', width=1000, height=700)
fig.show(renderer="colab")

In [26]:
top_authors = top_rated[:50]
fig = px.treemap(top_authors, path=['authors'], values='average_rating',title='Popular Authors', width=1000, height=700)
fig.show(renderer="colab")

### Bookcrossing EDA
- bookcrossing이란? 
    - 공공장소에 책을 남겨 놓아 다른 사람들도 그 책을 읽을 수 있도록 하는 일
- 책을 읽은 사람이 리뷰를 남겨놓고 점수를 준 데이터를 모아둔 것 같음 
    - 가장 많은 리뷰, 높은 점수 / 적은 리뷰, 적은 점수를 받은 책은 어떤 것이 있는지 알아보자!

데이터 셋 설명
```
1. Books
ISBN: book's unique id
Book-Title: Title of book
Book-Author: Author of book
Publisher: Organization that published book
Year-Of-Publication: published year
Image-URL-S: url of book's image small size
Image-URL-M: url of book's image medium size
Image-URL-L: url of book's image large size

2. Users
User-ID: unique id for user
Location: physicall address of user (city, state, country)
Age: age of this user

3. Book-Ratings:
User-ID: id of user who rated
ISBN: id of book that was rated
Book-Rating: score that user gives to book (from 1 to 10)
```

In [27]:
merge_data.head()

,isbn,book_title,book_author,year_of_publication,publisher,user_id,rating
626418,0000913154,the way things work an illustrated encyclopedi...,C. van Amerongen (translator),1967,Simon & Schuster,171118,8
587195,0001010565,mogs christmas,Judith Kerr,1992,Collins,209516,0
587194,0001010565,mogs christmas,Judith Kerr,1992,Collins,86123,0
441049,0001046713,twopence to cross the mersey,Helen Forrester,1992,HarperCollins Publishers,196149,0
263949,000104687X,ts eliot reading the wasteland and other poems,T.S. Eliot,1993,HarperCollins Publishers,23902,6


### Top Years of Publishing

In [28]:
# Top Years of Publishing
cross_typ = merge_data['year_of_publication'].value_counts().reset_index()
cross_typ.columns = ['value', 'count']
cross_typ['value'] = cross_typ['value'].astype(str) +' year'
cross_typ = cross_typ.sort_values('count')
fig = px.bar(cross_typ.tail(50), x="count", y="value", title='Top 20 Years of Publishing', orientation='h', color='value',
             width=1000, height=700)
fig.show(renderer="colab")

- 2000년 , 2002 ~ 2003년 출판된 책이 가장 많이 읽힘 

In [29]:
merge_data

,isbn,book_title,book_author,year_of_publication,publisher,user_id,rating
626418,0000913154,the way things work an illustrated encyclopedi...,C. van Amerongen (translator),1967,Simon & Schuster,171118,8
587195,0001010565,mogs christmas,Judith Kerr,1992,Collins,209516,0
587194,0001010565,mogs christmas,Judith Kerr,1992,Collins,86123,0
441049,0001046713,twopence to cross the mersey,Helen Forrester,1992,HarperCollins Publishers,196149,0
263949,000104687X,ts eliot reading the wasteland and other poems,T.S. Eliot,1993,HarperCollins Publishers,23902,6
...,...,...,...,...,...,...,...
413438,B0001FZGTO,ill let you go a novel,Bruce Wagner,2002,Villard,271705,0
575503,B0001I1KOG,new york public library literature companion,New York Public Library,2001,Free Press,148258,10
472829,B00029DGGO,good wife strikes back the,Elizabeth Buchan,0,Viking Adult,100088,0
543768,B0002JV9PY,the blockade runners,Jules Verne,0,Digireads.com,179791,0


In [30]:
# top authors(frequency of book)
cross_author = merge_data['book_author'].value_counts().reset_index()
cross_author.columns = ['value', 'count']

cross_author['value'] = cross_author['value'].astype(str)
cross_author = cross_author.sort_values('count')

fig = px.bar(cross_author.tail(50), x="count", y="value", title='Top 20 Authors', orientation='h', color='value',
             width=1000, height=700)
fig.show(renderer="colab")

In [31]:
top_book = merge_data['book_title'].value_counts().reset_index()
top_book.columns = ['value', 'count']
top_book['value'] = top_book['value']
top_book = top_book.sort_values('count')
fig = px.bar(top_book.tail(20), x="count", y="value", title='Top 20 Books', orientation='h', color='value',
             width=1000, height=700)
fig.show(renderer="colab")

In [32]:
# with publishers
by_publisher = merge_data.groupby('publisher')['isbn'].count()
by_publisher = by_publisher.sort_values(ascending = False)

figure = go.Figure(data = [go.Pie(labels = by_publisher.index[:10], 
                                  values = by_publisher.values,
                                  textinfo='label+percent',
                                  title= 'Ten publsihers with most published books.' 
                                 )])

fig.show(renderer="colab")

In [33]:
print(f'Ten most popular publishers published: {sum(by_publisher.values[:10])}, Others: {sum(by_publisher.values[11:])}')

Ten most popular publishers published: 131359, Others: 501928


In [34]:
# with book title
stopwords.words('english')
print(stopwords.words('english'))
# Here I want to split every title of book to words and check whereas this word length greater than 3 or it's not in stop words.
# If everything is fine, then I will add that word into array.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [35]:
from collections import Counter
results = Counter()
merge_data['book_title'].str.lower().str.split().apply(results.update)

626418    None
587195    None
587194    None
441049    None
263949    None
          ... 
413438    None
575503    None
472829    None
543768    None
543766    None
Name: book_title, Length: 641731, dtype: object

In [36]:
from tqdm import tqdm 
most_common = dict(results.most_common(1000))

to_replace = ['(', ')', '*', '((', '))']
own_stop_words = ['book', 'guide', 'novel']

top_words = {}

for k, v in tqdm(most_common.items()):
    
    for err in to_replace:
        k = k.replace(err, "")
        
    if (len(k) > 3) and (k not in stopwords.words()) and (k not in own_stop_words):
        top_words[k] = v

100%|██████████| 1000/1000 [00:06<00:00, 165.92it/s]


- 책의 모든 제목을 단어로 나누고 이 단어의 길이가 3보다 크고 stopword에 포함되지 않은 단어 수집 

In [37]:
# top_words, Sort dictionary by values
top_words = dict(sorted(top_words.items(), key=lambda item: item[1])[::-1][:20])

In [38]:
figure = go.Figure(data = [go.Pie(labels = list(top_words.keys()), 
                                  values = list(top_words.values()),
                                  textinfo='label+percent',
                                  title= 'Twenty most used words in book titles.' 
                                 )])

fig.show(renderer="colab")

- 미스테리, 시리즈, 생활, 클래식, 스토리 등등 가장 많이 사용되는 단어 20개를 시각화

# Recommendation System Using KNN 
### 필사 커널 
- https://www.kaggle.com/code/sunilgautam/recommendation-system-using-knn


## 협업 필터링 - KNN

### 협업 필터링이란? 
- 협업 필터링은 사용자와 아이템 간의 상호 상관 관계를 분석하여 새로운 사용자-아이템 관계를 찾아주는 것
- 사용자의 과거 경험과 행동 방식(User Behavior)에 의존하여 추천하는 시스템

In [39]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [40]:
books_data = pd.read_csv(f'{goodbooks_path}/books.csv')
ratings_data = pd.read_csv(f'{goodbooks_path}/ratings.csv')

In [41]:
pd.set_option('display.max_columns',None)

In [42]:
ratings_data.head()

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


### Clean the data

In [43]:
books_data = books_data.dropna()

In [44]:
ratings = ratings_data.sort_values("user_id")
ratings.drop_duplicates(subset=["user_id","book_id"], keep='first', inplace=True) 

books_data.drop_duplicates(subset='original_title', keep='first', inplace=True)

In [45]:
merged_df = pd.merge(books_data, ratings, how='left', left_on=['id'], right_on=['book_id'])
df = merged_df[['id','original_title', 'user_id', 'rating']]

df = df.rename(columns = {'id':'book_id'})
df.head(200)

,book_id,original_title,user_id,rating
0,1,The Hunger Games,314,5
1,1,The Hunger Games,439,3
2,1,The Hunger Games,588,5
3,1,The Hunger Games,1169,4
4,1,The Hunger Games,1185,4
...,...,...,...,...
195,2,Harry Potter and the Philosopher's Stone,51166,5
196,2,Harry Potter and the Philosopher's Stone,51460,4
197,2,Harry Potter and the Philosopher's Stone,51838,4
198,2,Harry Potter and the Philosopher's Stone,52036,3


In [46]:
df.describe()

,book_id,user_id,rating
count,763421.000000,763421.000000,763421.000000
mean,4657.948579,25132.161966,3.852257
std,2875.106491,15229.803682,0.980928
min,1.000000,1.000000,1.000000
25%,2123.000000,11843.000000,3.000000
50%,4509.000000,24230.000000,4.000000
75%,7087.000000,38032.000000,5.000000
max,9999.000000,53424.000000,5.000000


In [55]:
ratings_df = df.pivot_table(index='book_id', columns ='user_id', values='rating').fillna(0)
pd.set_option('display.max_columns', 100)
ratings_df.head()

user_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,53375,53376,53377,53378,53379,53380,53381,53382,53383,53384,53385,53386,53387,53388,53389,53390,53391,53392,53393,53394,53395,53396,53397,53398,53399,53400,53401,53402,53403,53404,53405,53406,53407,53408,53409,53410,53411,53412,53413,53414,53415,53416,53417,53418,53419,53420,53421,53422,53423,53424
book_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
ratings_df.shape

(50681, 7755)

## KNN 협업 필터링 
KNN(K-Nearest Neighbor)
K명의 최근접 이웃에 기반해서 찾는 방법으로, 널리 쓰이는 방법이다.    
사용자가 준 평점으로 유사한 사람의 아이템을 찾거나, 유사한 아이템을 찾아 추천을 한다.     
편향을 제거(전반적으로 평점을 후하게 주거나 적게 주는 경우를 방지)해주기 위해 비교군의 평점을 더해주거나 빼주어 동일하게 해준다.    

추천 리스트에 새로운 사용자 또는 아이템이 오더라도 안정정으로 진행이 가능하다.    
방법이 간단하고 직관적이어서 접근이 용이하다.    
유저 기반의 방법 및 속도, 메모리가 많이 든다.     
희소성으로 인한 제약이 발생한다. (유사한 이웃이 사용한 경험이 없으면 추천 불가능하다)     


### 알고리즘 구현 

아이템 id와 사용자 id를 행과 열로 하는 매트릭스를 생성하고,  코사인 유사도를 이용해 특정 사용자 가장 유사한 사용자를 찾아낸다. 

#### 코사인 유사도 공부 

In [49]:
# 샘플 코드 
# 여기서 코사인 유사도를 이용해 특정한 사용자와와 가장 유사한 사용자를 찾아낼 것이다.

def cosine_simillarity(v1, v2):
  A = np.sqrt(np.sum(np.square(v1)))
  B = np.sqrt(np.sum(np.square(v2)))
  return np.dot(v1,v2) / (A*B)

In [50]:
import random
print(random.randint(1,53424))

38587


In [56]:
# ratings_df.to_numpy()

In [57]:
# # 특정 사용자의 코사인 유사도

# sim_lst = []  # 빈 리스트 생성
# best_score = 0
# mat = ratings_df.to_numpy()
# for idx, vec in enumerate(mat):  # 매트릭스의 각 사용자 별 벡터를 뽑아 vec에 넣기

#     similarity = cosine_simillarity( vec, mat[51166]) # 사용자23568의 벡터
#     sim_lst.append((idx,similarity))  
#     if idx != 2 and best_score < similarity:  # 현재 계산한 유사도가 기존 최고 유사도보다 높다면 바꿔준다. 
#       best_score = similarity
#       best_user = idx +1   
# print(sim_lst, f"\n사용자 23568과 가장 비슷한 유저: 사용자{best_user} \n유사도:{best_score}")

#### KNN
- https://scikit-learn.org/stable/modules/neighbors.html

In [58]:
ratings_matrix = csr_matrix(ratings_df.values)

 'algorithm', which must be one of ['auto', 'ball_tree', 'kd_tree', 'brute']

In [60]:
model_knn = NearestNeighbors(metric='cosine', algorithm = 'brute')
model_knn.fit(ratings_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [61]:
def get_book_id(book_title):
    target_df = df.loc[df['original_title'] == book_title]
    return target_df['book_id'].iloc[0]

id_TheHungerGames = get_book_id('The Hunger Games')
print(id_TheHungerGames)

1


In [62]:
def get_title(book_id):
    target_df = df.loc[df['book_id'] == book_id]
    return target_df['original_title'].iloc[0]

print(get_title(1))

The Hunger Games


In [80]:
def get_recomm(book_title, num_neighbors=10, display=False): 
    book_ids = []
    distances, indices = None, None
    
    query_index = get_book_id(book_title) - 1
    print( distances, indices)
    if num_neighbors > 0:
        distances, indices = model_knn.kneighbors(ratings_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = num_neighbors + 1)
    else:
        distances, indices = model_knn.kneighbors(ratings_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 10 + 1)
    
    for i in range(0, len(distances.flatten())):
        if display is True:
            if i == 0:
                print('Recommendations for ', book_title, '\n')
            else:    
                id = ratings_df.index[indices.flatten()[i]]
                
                print(f'{i}\t Book ID: {id} \tBook Title : {get_title(id)} \tDistance: {distances.flatten()[i]:.2f} \n') 
        book_ids.append(ratings_df.index[indices.flatten()[i]])
    
    return book_ids

In [81]:
recommendations_for_TheHungerGames = get_recomm('The Hunger Games', num_neighbors=10, display=True)

None None
Recommendations for  The Hunger Games 

1	 Book ID: 17 	Book Title : Catching Fire 	Distance: 0.41 

2	 Book ID: 31 	Book Title : The Help 	Distance: 0.43 

3	 Book ID: 2 	Book Title : Harry Potter and the Philosopher's Stone 	Distance: 0.44 

4	 Book ID: 20 	Book Title : Mockingjay 	Distance: 0.45 

5	 Book ID: 3 	Book Title : Twilight 	Distance: 0.49 

6	 Book ID: 93 	Book Title : The Secret Garden 	Distance: 0.51 

7	 Book ID: 5 	Book Title : The Great Gatsby 	Distance: 0.52 

8	 Book ID: 16 	Book Title : Män som hatar kvinnor 	Distance: 0.52 

9	 Book ID: 9 	Book Title : Angels & Demons  	Distance: 0.53 

10	 Book ID: 37 	Book Title : The Lion, the Witch and the Wardrobe 	Distance: 0.53 

